In [9]:
import os
import re
import glob
import sys
import pandas as pd
from datetime import datetime
import numpy as np
from pandas.compat import StringIO

PATH = '/Users/Inmetrics/Documents/Sascar/4 - Coleta/BateriaDeExecucao/postgres/ps44/'
LOGFILE='*stat_statements.txt'
#'teste.txt' #' '1403_1008_Teste.txt' #'
filesRead = glob.glob(PATH + LOGFILE)  
fileRead = []
df_metricas = pd.DataFrame()
contadorlinha = -1


In [10]:
#Métodos de Trampo sujo

def coletarDataHora(nomeArquivo):
    valor = re.findall("\d\d\d\d\S\d\d\d\d", nomeArquivo)
    date = datetime(2018,int(valor[0][5:7]),int(valor[0][7:9]),int(valor[0][0:2]),int(valor[0][2:4]),0)
    return date

def linhaMais(linha):
    valor = re.findall("^(\s+\S){3}", linha)
    if valor:
        return True
    return False

def processarlinha(linha):
    cond1 = re.findall("^.+from pg_stat_statements;", linha)
    cond2 = re.findall("^.+userid", linha)
    cond3 = re.findall("^--------", linha)
    cond4 = re.findall("^.+rows\W", linha)
    cond5 = re.findall("^\s+$", linha)
    if cond1 or cond2 or cond3 or cond4 or cond5:
        return False
    return True

def incluirLinha(line, df_metricas):
    #df_metricas = pd.DataFrame()
    line = line.replace('||','+') 
    df = pd.read_csv(StringIO(line), sep="|", names=['userid','dbid', 'queryid','query', 'calls', 'total_time',
                                                               'min_time', 'max_time', 'mean_time','stddev_time', 'rows', 'shared_blks_hit', 
                                                               'shared_blks_read','shared_blks_dirtied','shared_blks_written', 'local_blks_hit',
                                                               'local_blks_read', 'local_blks_dirtied','local_blks_written','temp_blks_read',
                                                               'temp_blks_written','blk_read_time','blk_write_time', 'Datahora'])
    df_metricas = pd.concat([df_metricas,df])
    return df_metricas

In [11]:
### REalizar parse
def efetuarParse(arquivo):
    global contadorlinha
    global df_metricas
    fileRead = open(arquivo, "r", encoding='utf-8')
    datahora = coletarDataHora(arquivo)
    for line in fileRead:
        if (linhaMais(line)):
            line = line.replace('|',' ') 
            df_metricas.iloc[contadorlinha , 3] = str(df_metricas.iloc[contadorlinha , 3]) + line
        else:
            if processarlinha(line):
                df_metricas = incluirLinha(line, df_metricas)
                contadorlinha +=1
                df_metricas.iloc[contadorlinha , 23] =datahora
    fileRead.close()
    return df_metricas
          


In [12]:
for fileLog in filesRead:
    df_metricas = efetuarParse(fileLog)

writer = pd.ExcelWriter('/Users/Inmetrics/Documents/Sascar/Consultas172_19_12_44_stat_statements.xlsx' , engine='xlsxwriter')
df_metricas.to_excel(writer, sheet_name='Plan1')
writer.save()

KeyboardInterrupt: 